# Defining Classes

In [ ]:
doremi_dir = "/kaggle/input/doremi-raw-dataset/DoReMi_v1/"

In [ ]:
class Staff:
    def __init__(self, center: float, upper_bound: int, lower_bound: int, unit_size: float, track: int):
        self.center = center
        self.upper_bound = upper_bound
        self.lower_bound = lower_bound
        self.unit_size = unit_size
        self.track = track

    def __repr__(self):
        return (
            "Staff {\n"
            f"    Center: {self.center:.1f}\n"
            f"    Upper bound: {self.upper_bound}\n"
            f"    Lower bound: {self.lower_bound}\n"
            f"    Unit size: {self.unit_size:.1f}\n"
            f"    Track: {self.track}\n"
            "}"
        )
        
class Note:
    def __init__(self, class_name: str, top: int, left: int, width: int, height: int, img_name: str):
        self.class_name = class_name
        self.top = top
        self.left = left
        self.width = width
        self.height = height
        self.image = img_name
    def gather_coordinates(self):
        return [self.top, self.left, self.width, self.height]
    def create_box(self):
        return [(self.left, self.top), (self.left + self.width, self.top + self.height)]

# Preprocessing Images

## Sample Image

In [ ]:
from PIL import Image

img_path = doremi_dir + 'Images/Delius - String Quartet mvt III-003.png'
image = Image.open(img_path)
file_path = doremi_dir + 'Parsed_by_page_omr_xml/Parsed_Alkan - Posement-layout-0-muscima_Page_1.xml'
with open(file_path, 'r') as file:
    content = file.read(200)
    print(content)
image

## Preprocessing Stafflines

In [ ]:
import re
import pprint
import os

def natural_key(string):
    return [int(s) if s.isdigit() else s.lower() for s in re.split('(\d+)', string)]

# Sorts Images and Parsed_by_page_omr_xml directories
image_list = os.listdir(doremi_dir + "Images")
omr_xml_list =  os.listdir(doremi_dir + "Parsed_by_page_omr_xml")

image_list.sort(key=natural_key)
omr_xml_list.sort(key=natural_key)

# Matches each .png file to their respective omr_xml file
image_to_omr = {img: omr for img, omr in zip(image_list, omr_xml_list)}

def extract_staffs_from_image(cropped_image, padding_multiplier=4.5):
    image_gray = cropped_image.convert("L")
    image_np = np.array(image_gray)
    _, binary = cv2.threshold(image_np, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    horizontal_projection = np.sum(binary, axis=1)
    peaks, _ = find_peaks(horizontal_projection, height=np.max(horizontal_projection) * 0.5, distance=5)

    grouped_staffs = []
    for i in range(0, len(peaks) - 4, 5):
        group = peaks[i:i+5]
        grouped_staffs.append(group)

    staff_objects = []
    for i, group in enumerate(grouped_staffs):
        y_center = float(np.mean(group))
        unit_size = float(np.mean(np.diff(group)))
        padding = int(unit_size * padding_multiplier)

        y_min = int(np.min(group) - padding)
        y_max = int(np.max(group) + padding)
        track = i % 2 + 1

        staff = Staff(
            center=y_center,
            upper_bound=y_min,
            lower_bound=y_max,
            unit_size=unit_size,
            track=track
        )
        staff_objects.append(staff)

    return staff_objects

In [ ]:
from PIL import Image
import os
import numpy as np
import cv2 
from scipy.signal import find_peaks

images = []
staff_data_per_image = {}

selected_files = [
    "Delius - String Quartet mvt III",
    "Schumann - String Quartet 1 mvt 3",
    "Webern - Variations Op 27 III"]

for image_name in image_list:
    if not any(image_name.startswith(prefix) for prefix in selected_files):
        continue

    image_path = os.path.join(doremi_dir, "Images", image_name)
    try:
        cropped_image = Image.open(image_path)
        staff_objects = extract_staffs_from_image(cropped_image)

        images.append(image_name)
        staff_data_per_image[image_name] = staff_objects 

        if len(images) == 200:
            break
    except Exception as e:
        print(f"Skipping {image_name} due to error: {e}")
        continue

# Confirm results
print(f"\nFound {len(images)} images.\n") 
print(images)

In [ ]:
max_height = 0
max_image_name = None
max_upper = None
max_lower = None

IMG_TO_STAFF = {}

for image_name in images:
    image_path = os.path.join(doremi_dir + "Images", image_name)
    img = Image.open(image_path)

    staff_objects = extract_staffs_from_image(img)
    n = len(staff_objects)
    if max_upper == None or staff_objects[0].upper_bound < max_upper:
        max_upper = staff_objects[0].upper_bound
        
    if max_lower == None or staff_objects[n - 1].lower_bound > max_lower:
        max_lower = staff_objects[n - 1].lower_bound

    IMG_TO_STAFF[image_name] = staff_objects

print(f"Upper bound: {max_upper}")
print(f"Lower bound: {max_lower}")

In [ ]:
import cv2
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt

image_gray = image.convert("L")
image_np = np.array(image_gray)

_, binary = cv2.threshold(image_np, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

horizontal_projection = np.sum(binary, axis=1)

peaks, _ = find_peaks(horizontal_projection, height=np.max(horizontal_projection) * 0.5, distance=5)

grouped_staffs = []
for i in range(0, len(peaks) - 4, 5):
    group = peaks[i:i+5]
    grouped_staffs.append(group)

image_color = np.stack([image_np]*3, axis=-1)

staff_bounds = []
staff_bounds = []
staff_objects = []

for i, group in enumerate(grouped_staffs):
    y_center = float(np.mean(group))
    unit_size = float(np.mean(np.diff(group)))
    padding = int(unit_size * 3.5)

    y_min = int(np.min(group) - padding)
    y_max = int(np.max(group) + padding)
    track = i % 2 + 1

    staff = Staff(
        center=y_center,
        upper_bound=y_min,
        lower_bound=y_max,
        unit_size=unit_size,
        track=track
    )
    staff_objects.append(staff)
    staff_bounds.append((y_min, y_max))

    cv2.rectangle(image_color, (0, y_min), (image_np.shape[1], y_max), (0, 255, 0), 2)
    print(staff)


plt.figure(figsize=(20, 20))
plt.imshow(image_color)
plt.axis("off")
plt.show()

## Preprocessing Notes

In [ ]:
import xml.etree.ElementTree as ET

IMAGE_TO_NOTES = {}
for key in images:
# Parse the XML file
    xml_path = os.path.join(doremi_dir, "Parsed_by_page_omr_xml", image_to_omr[key])
    tree = ET.parse(xml_path) 
    root = tree.getroot()
# Define target classnames
    target_classes = {'noteheadBlack', 'noteheadHalf', 'noteheadWhole'}

# Collect matching nodes
    matching_nodes = []

# Loop through all <Node> elements
    for node in root.findall('.//Node'):
        class_name = node.find('ClassName')
        if class_name is not None and class_name.text in target_classes:
            matching_nodes.append(node)

# Print results
    final_notes = []
    for match in matching_nodes:
        note = Note(match.find("ClassName").text, 
                    int(match.find("Top").text), 
                    int(match.find("Left").text), 
                    int(match.find("Width").text), 
                    int(match.find("Height").text), 
                    key
                   )
        final_notes.append(note)
        # print(note.image)
    IMAGE_TO_NOTES[key] = final_notes

### Cropping Stafflines

In [ ]:
CROPPED_NOTE_PAIRS = []
for image_name, notes in IMAGE_TO_NOTES.items():
    image_path = os.path.join(doremi_dir + "Images", image_name)
    img = Image.open(image_path)
    width = img.width
    
    # Crop image to vertical slice (top to bottom)
    cropped = img.crop((0, max_upper, width, max_lower))
    for note in notes:
        note.top -= max_upper
    CROPPED_NOTE_PAIRS.append((cropped, notes))

In [ ]:
from PIL import Image, ImageDraw

for i in range(5):
    test_image, notes = CROPPED_NOTE_PAIRS[i]
    img = test_image.copy()  
    draw = ImageDraw.Draw(img)

    # Draw bounding boxes on the image
    counter = 0;
    for note in notes:
        draw.rectangle(note.create_box(), outline="red", width=2)
        draw.text(note.create_box()[1], str(counter), fill="black")
        counter += 1
    display(img)
    width, height = img.size
    print(width)
    print(height)
        
test_image, notes = CROPPED_NOTE_PAIRS[0]
img = test_image.copy()  
sample_cropped_img = test_image

# Extracting Note

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt 

def remove_staff_lines(binary_img: np.ndarray) -> np.ndarray:
    """
    Detect and remove horizontal staff lines from the binary image input. 

    Output: 
        np.ndarray: a binary image with horizontal staff lines removed.
    """
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40,1))
    detected_stafflines = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN, horizontal_kernel, iterations=1)
    return cv2.subtract(binary_img, detected_stafflines)

In [ ]:
import cv2
import numpy as np

def selective_hole_fill(binary, min_hole_area, max_hole_area) -> np.ndarray:
    """
    Fill only fully enclosed holes whose area is between min_hole_area and max_hole_area.
    """
    # invert the binary image
    inv = cv2.bitwise_not(binary)
    
    # label connected components in inverted image
    num, labels, stats, centroids = cv2.connectedComponentsWithStats(inv, connectivity=8)
    
    h, w = binary.shape
    filled = binary.copy()

    # filling step
    for lbl in range(1, num): # iterated thru each component
        area = stats[lbl, cv2.CC_STAT_AREA] 
        # only fill holes in the desired size range
        if area < min_hole_area or area > max_hole_area:
            continue
        filled[labels == lbl] = 255

    return filled

In [ ]:
WIDTH_FACTOR = 0.5
HEIGHT_FACTOR = 0.4
MIN_AREA = 100 # pixels
MAX_AREA = 600

MAX_HEIGHT_WIDTH_RATIO = 2.0 # h / w
MIN_WIDTH_HEIGHT_RATIO = 1 # w / h 
ELLIPSE_SIZE = 20.5
PAD = 4

def extract_noteheads(img, debug: bool) -> np.ndarray:
    # testing with 'cropped_img' variable
    gray_img = img.convert('L')
    gray_img = np.array(gray_img, dtype=np.uint8)

    # assign all pixel values higher than 127 to 255 and others to 0
    _, binary_img = cv2.threshold(gray_img, 127, 255, cv2.THRESH_BINARY_INV)
    
    if debug:
        plt.figure(figsize=(6,6))
        plt.imshow(binary_img, cmap='gray')
        plt.title("Binary Inverted")
        plt.axis('off')
        plt.show()
        
    # remove staff lines
    processed_img = remove_staff_lines(binary_img)

    # Performing morphological operations
    w_close = max(1, int(round(ELLIPSE_SIZE * 0.2)))
    h_close = max(1, int(round(ELLIPSE_SIZE * 0.4)))
    close_krn = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (w_close, h_close))
    closed = cv2.morphologyEx(processed_img, cv2.MORPH_CLOSE, close_krn)
    
    filled = selective_hole_fill(
        closed, 
        min_hole_area=150,
        max_hole_area=250,
        )
    
    if debug:
        plt.figure(figsize=(6,6))
        plt.imshow(closed, cmap='gray')
        plt.title("After Closed")
        plt.axis('off')
        plt.show()
           
    notehead_size = (
        int(round(ELLIPSE_SIZE * WIDTH_FACTOR)),
        int(round(ELLIPSE_SIZE * HEIGHT_FACTOR))
    )
    
    # erode away isolated pixels that are not note heads
    kernel_2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, notehead_size)
    opened = cv2.morphologyEx(filled, cv2.MORPH_OPEN, kernel_2)

    if debug:
        plt.figure(figsize=(6,6))
        plt.imshow(opened, cmap='gray')
        plt.title("After Opened")
        plt.axis('off')
        plt.show()
        
    w_close = max(1, int(round(ELLIPSE_SIZE * 0.1)))
    h_close = max(1, int(round(ELLIPSE_SIZE * 0.8)))
    close_krn = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (w_close, h_close))
    closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, close_krn)
    
    if debug:
        plt.figure(figsize=(6,6))
        plt.imshow(closed, cmap='gray')
        plt.title("After Second Closed")
        plt.axis('off')
        plt.show()
        
    # find contours
    contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if debug:
        canvas = cv2.cvtColor(closed, cv2.COLOR_GRAY2BGR)
        
        cv2.drawContours(canvas, contours, -1, (0,255,0), 1)
        
        plt.figure(figsize=(6,6))
        plt.imshow(canvas[...,::-1])
        plt.title(f'{len(contours)} contours found')
        plt.axis('off')
        plt.show()

    # filter out shapes that are not noteheads by checking how round the white area is
    notehead_mask = np.zeros_like(opened)
    for item in contours:
        area = cv2.contourArea(item)
        # print(area)
        if area < MIN_AREA or area > MAX_AREA:
            continue
        
        x, y, w, h = cv2.boundingRect(item)
        
        # Drop any blob that's too tall/thin:
        if h / float(w) > MAX_HEIGHT_WIDTH_RATIO:
            continue

        # Drop any blob that's too wide/flat:
        if w / float(h) < MIN_WIDTH_HEIGHT_RATIO:
            continue

        cv2.drawContours(notehead_mask, [item], -1, 255, -1)

    morphed = notehead_mask

    if debug:
        plt.figure(figsize=(6,6))
        plt.imshow(morphed, cmap='gray')
        plt.title("Morphed (staff lines out, noteheads closed)")
        plt.axis('off')
        plt.show()
        
    final_contours, _ = cv2.findContours(
        notehead_mask,
        cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE
    )

    return morphed, final_contours

In [ ]:
test_image, notes = CROPPED_NOTE_PAIRS[5]
morphed_img, contours = extract_noteheads(test_image, True)

In [ ]:
def display_image(image, title="Image"):
    plt.figure(figsize=(10, 8))
    plt.imshow(image, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

In [ ]:
def get_roi_coords(mask):
    """
    Compute padded bounding-box coordinates for a binary mask.
    Returns (x0, y0, x1, y1).
    """
    x, y, w, h = cv2.boundingRect(mask)
    # make sure we don't go out of bounds
    x0 = max(0, x - PAD) 
    y0 = max(0, y - PAD)
    x1 = min(mask.shape[1], x + w + PAD)
    y1 = min(mask.shape[0], y + h + PAD)
    return x0, y0, x1, y1

def split_horizontal_blob(mask, debug=False):
    """
    Split a single-blob mask horizontally through its vertical midpoint.
    Returns [top_half, bottom_half] or [mask] if one half is empty.
    """
    # Compute bounding box
    x, y, w, h = cv2.boundingRect(mask)
    mid_y = y + h // 2 # calculating the midway point

    # Empty masks
    top = np.zeros_like(mask)
    bottom = np.zeros_like(mask)

    # Slicing within bbox
    top[y:mid_y, x:x+w] = mask[y:mid_y, x:x+w]
    bottom[mid_y:y+h, x:x+w] = mask[mid_y:y+h, x:x+w]
    pieces = [top, bottom]

    # If one half is empty, return original. This means that either mask is completely all black
    if cv2.countNonZero(top) == 0 or cv2.countNonZero(bottom) == 0:
        return [mask]

    # Creating a kernel
    radius = 5
    ksize = 2 * radius + 1
    kern  = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (ksize, ksize))

    # apply opening to each piece to erode straight edges, then dilate back
    # basically erosding away the straight edges and then dilating again to give it its shape back
    rounded = [cv2.morphologyEx(p, cv2.MORPH_OPEN, kern) for p in pieces]

    if debug:
        for i, m in enumerate(rounded):
            x0, y0, x1, y1 = get_roi_coords(mask)
            plt.figure(figsize=(3,3))
            plt.imshow(m[y0:y1, x0:x1], cmap='gray')
            plt.title(f'Rounded Piece #{i}')
            plt.axis('off')
        plt.show()

    return rounded

In [ ]:
def split_blob(mask, debug=False):
    """
    Split a large binary mask into smaller ones using watershed,until each piece's nonzero area <= MAX_AREA.
    Returns a list of mask pieces.
    """
    # If this mask is already small enough, return it. Error checking bascially
    area = cv2.countNonZero(mask)
    if area <= MAX_AREA:
        if debug:
            x0, y0, x1, y1 = get_roi_coords(mask)
            plt.figure(figsize=(4,4))
            plt.imshow(mask[y0:y1, x0:x1], cmap='gray', vmin=0, vmax=255, interpolation='nearest')
            plt.title(f'Final Piece (area={area})')
            plt.axis('off')
        return [mask]

    # 1) Compute distance transform
    # Computes shortest distance from foreground pixel to the background pixel. Identifying the center regions of the object
    dist = cv2.distanceTransform(mask, cv2.DIST_L2, 5)
    if debug:
        x0, y0, x1, y1 = get_roi_coords(mask)

        plt.figure(figsize=(4,4))
        plt.imshow(dist[y0:y1, x0:x1], cmap='viridis', interpolation='nearest')
        plt.title('Distance Transform')
        plt.axis('off')

    # 2) Get sure foreground peaks (areas that are confidently park of the foreground, deep enough within the object)
    # Extracting deepest pixels from background boundary, 40% of blob distance
    thresh_val = 0.6 * dist.max()
    _, fg = cv2.threshold(dist, thresh_val, 255, cv2.THRESH_BINARY)
    fg = fg.astype(np.uint8) # converting to be used in OpenCV
    if debug:
        x0, y0, x1, y1 = get_roi_coords(mask)

        plt.figure(figsize=(4,4))
        plt.imshow(fg[y0:y1, x0:x1], cmap='gray', vmin=0, vmax=255, interpolation='nearest')
        plt.title('Sure Foreground Peaks')
        plt.axis('off')

    # 3) Create markers for watershed
    num_components, markers = cv2.connectedComponents(fg)
    if num_components < 2: # only one component in the foreground
        return [mask]
    markers = markers + 1 # 0 is reserved for the background
    unknown = cv2.subtract(mask, fg) # marking the unknown region
    markers[unknown == 255] = 0 # uncertain pixels
    
    if debug:
        x0, y0, x1, y1 = get_roi_coords(mask)

        plt.figure(figsize=(4,4))
        plt.imshow(markers[y0:y1, x0:x1], cmap='tab20', interpolation='nearest')
        plt.title('Watershed Markers')
        plt.axis('off')

    # 4) Apply watershed
    color = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR) # watershed needs a colored image
    cv2.watershed(color, markers) # running watershed, flooding from seed labels
    
    if debug:
        boundary_overlay = color.copy()
        boundary_overlay[markers == -1] = (0,0,255)
        x0, y0, x1, y1 = get_roi_coords(mask)

        plt.figure(figsize=(4,4))
        plt.imshow(boundary_overlay[y0:y1, x0:x1][..., ::-1], interpolation='nearest')
        plt.title('Watershed Boundaries ')
        plt.axis('off')

    # 5) Extract pieces
    pieces = []
    for lbl in range(2, num_components+1):
        piece = np.zeros_like(mask)
        piece[markers == lbl] = 255
        if cv2.countNonZero(piece) > 0:
            if debug:
                x0, y0, x1, y1 = get_roi_coords(piece)
                plt.figure(figsize=(4,4))
                plt.imshow(piece[y0:y1, x0:x1], cmap='gray', vmin=0, vmax=255, interpolation='nearest')
                plt.title(f'Piece {lbl-1}')
                plt.axis('off')
            pieces.append(piece)

    # If watershed didn't split, return original
    if not pieces:
        return [mask]
        
    aspect_thresh = 1.2 # height has to be 120% of width, if its greater, that means there are two notes on top of each other
    pieces = []
    
    for lbl in range(2, num_components+1):
        piece = np.zeros_like(mask)
        piece[markers == lbl] = 255
        if cv2.countNonZero(piece) > 0:
            # checking aspect ratio
            x,y,w,h = cv2.boundingRect(piece)
                
            if h > w * aspect_thresh or h * aspect_thresh > w:
                if True:
                    x0, y0, x1, y1 = get_roi_coords(piece)
                    plt.figure(figsize=(4,4))
                    plt.imshow(piece[y0:y1, x0:x1], cmap='gray', vmin=0, vmax=255, interpolation='nearest')
                    plt.title(f'Piece {lbl-1} height: {h}, width: {w}')
                    plt.axis('off')
                # split horizontally if its too tall or too wide
                sub = split_horizontal_blob(piece, True)
                pieces.extend(sub)
            else:
                if debug:
                    x0, y0, x1, y1 = get_roi_coords(piece)
                    plt.figure(figsize=(4,4))
                    plt.imshow(piece[y0:y1, x0:x1], cmap='gray', vmin=0, vmax=255, interpolation='nearest')
                    plt.title(f'Piece {lbl-1} (zoomed)'); plt.axis('off')
                pieces.append(piece)
    return pieces

In [ ]:
import cv2
import numpy as np
id_num = 0
def refine_and_fit_ellipses(gray_img, notehead_mask, contours,
                            iou_thresh=0.75,
                            min_area=150,
                            min_ellipse_area=300,
                            min_minor_axis=10,
                            min_axis_ratio=0.5,
                            min_area_ratio=0.6,
                            MAX_AREA_ratio=1.3,
                            min_solidity=0.9,
                            debug=False,):
    """
    Returns two lists:
     - accepted: [( (cx,cy), (MA,ma), angle ), ... ]
     - rejected: [ ( (cx,cy), reason_string ), ... ]
    """
    H, W = gray_img.shape
    accepted, rejected = [], []
    global id_num
    for cnt in contours:
        # Computing the raw area
        rawA = cv2.contourArea(cnt)
        blob = np.zeros_like(notehead_mask)
        cv2.drawContours(blob, [cnt], -1, 255, -1)
        
        if debug:
            x0, y0, x1, y1 = get_roi_coords(blob)
            
            plt.figure(figsize=(4,4))
            plt.imshow(blob[y0:y1, x0:x1],
                       cmap='gray',
                       vmin=0, vmax=255,
                       interpolation='nearest')
            plt.title("Blob mask")
            plt.axis('off')

        # If raw area is bigger than the max area, split into multple blobs
        if rawA > MAX_AREA:
            submasks = split_blob(blob, False)
        else:
            submasks = [blob]

        for piece in submasks:
            # extracting pieces from the submask and adding PADding 
            x0, y0, x1, y1 = get_roi_coords(piece)

            roi_gray = gray_img[y0:y1, x0:x1]
            roi_mask = piece[y0:y1, x0:x1]
            
            if debug:
                plt.figure(figsize=(4,4))
                plt.imshow(roi_gray,
                           cmap='gray',
                           vmin=0, vmax=255,
                           interpolation='nearest')
                plt.title("ROI gray")
                plt.axis('off')
                
                plt.figure(figsize=(4,4))
                plt.imshow(roi_mask,
                           cmap='gray',
                           vmin=0, vmax=255,
                           interpolation='nearest')
                plt.title("ROI mask")
                plt.axis('off')
                
            # finding the local contours within the mask
            local_cnts, _ = cv2.findContours(
            roi_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
            )

            # Run 9 total tests
            for lc in local_cnts:
                area = cv2.contourArea(lc)
                cx_loc, cy_loc = cv2.boundingRect(lc)[:2]
                (cx_loc, cy_loc), (MA, ma), ang = cv2.fitEllipse(lc)
                note_metadata = ((cx_loc+x0, cy_loc+y0),(MA,ma),ang)
                
                # 1) raw-area floor
                if area < min_area or area > MAX_AREA:
                    rejected.append(note_metadata)
                    continue

                # 2) need at least 5 points to fit an ellipse
                if len(lc) < 5:
                    rejected.append(note_metadata)
                    continue

                # 4) ellipse-area floor
                ell_area = np.pi*(MA/2)*(ma/2)
                if ell_area < min_ellipse_area:
                    rejected.append(note_metadata)
                    continue

                # 5) minor-axis floor
                if min(MA, ma) < min_minor_axis:
                    rejected.append(note_metadata)
                    continue

                # 6) axis-ratio
                axis_r = min(MA,ma)/max(MA,ma)
                if axis_r < min_axis_ratio:
                    rejected.append(note_metadata)
                    continue

                # 7) area-ratio
                area_r = area/ell_area
                if not (min_area_ratio <= area_r <= MAX_AREA_ratio):
                    rejected.append(note_metadata)
                    continue

                # if we get here, it’s a keeper
                accepted.append((note_metadata, id_num))
                id_num+=1

    return accepted, rejected

In [ ]:
from tqdm import tqdm
debug = False
ACCEPTED_NOTES = []
for test_img, notes in tqdm(CROPPED_NOTE_PAIRS, desc="Extracting noteheads", unit="pair"):    
    # 1) Make image gray & binary
    if not isinstance(test_img, np.ndarray):
        gray = np.array(test_img.convert('L'), dtype=np.uint8)
    else:
        gray = test_img.astype(np.uint8)
    
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    
    # 2) Extract noteheads
    morphed, contours = extract_noteheads(
        test_img,
        debug=False
    )
    
    # 3) Visualize the cleaned contours
    if debug:
        canvas = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)
        cv2.drawContours(canvas, contours, -1, (0,255,0), 2)
        plt.figure(figsize=(6,6))
        plt.imshow(canvas[..., ::-1])
        plt.title(f"Cleaned Image {i}: {len(contours)} contours")
        plt.axis('off')
        plt.show()
        
    # 4) Now call the new fitter
    accepted, rejected = refine_and_fit_ellipses(
        gray_img = gray,
        notehead_mask = morphed,
        contours = contours,
        min_area = MIN_AREA,
        debug = False
    )
    ACCEPTED_NOTES.append((gray, accepted))
    if debug:
        print(f"Accepted: {len(accepted)} ellipses")
        print(f"Rejected: {len(rejected)} blobs")
        
        # 5) Overlay the kept ellipses on your original
        canvas2 = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
        for (cx,cy), (MA,ma), ang in accepted:
            cv2.ellipse(
                canvas2,
                (int(round(cx)), int(round(cy))),
                (int(round(MA/2)), int(round(ma/2))),
                angle=ang,
                startAngle=0,
                endAngle=360,
                color=(0,0,255),
                thickness=2
            )
        
        plt.figure(figsize=(8,8), dpi=200)
        plt.imshow(canvas2[..., ::-1])
        plt.title(f"Accepted Ellipses: {len(accepted)}")
        plt.axis('off')
        plt.show()
         
        canvas2 = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
        for (cx,cy), (MA,ma), ang in rejected:
            cv2.ellipse(
                canvas2,
                (int(round(cx)), int(round(cy))),
                (int(round(MA/2)), int(round(ma/2))),
                angle=ang,
                startAngle=0,
                endAngle=360,
                color=(0,0,255),
                thickness=2
            )
            
        plt.figure(figsize=(8,8), dpi=200)
        plt.imshow(canvas2[..., ::-1])
        plt.title(f"Rejected Ellipses: {len(rejected)}")
        plt.axis('off')
        plt.show()
    

In [ ]:
debug = False
TARGET_SIZE = (39, 23)

def crop_pad_notes(gray_img, accepted, debug = False):
    H, W = gray_img.shape[:2]
    bboxes = []
    for ((cx, cy), (MA, ma), _), id_num in accepted:
        # center X, center Y
        # Major Axis - width
        # Minor axis - height
        w = int(round(MA))
        h = int(round(ma))
        x0 = max(0, int(round(cx - w/2)))
        y0 = max(0, int(round(cy - h/2)))
        x1 = min(W, int(round(cx + w/2)))
        y1 = min(H, int(round(cy + h/2)))
        bboxes.append(((x0, y0, x1, y1), id_num))

    # finding universal size
    IMAGE_WIDTH, IMAGE_HEIGHT = 0,0
    for (x0,y0,x1,y1), _ in bboxes:
        IMAGE_WIDTH = max(x1 - x0, IMAGE_WIDTH)
        IMAGE_HEIGHT = max(y1 - y0, IMAGE_HEIGHT)

    if debug:
        print(f"IMAGE_WIDTH: {IMAGE_WIDTH} IMAGE_HEIGHT: {IMAGE_HEIGHT}")
    
    padded_imgs = []
    for (x0,y0,x1,y1), id_num in bboxes:
        cropped = gray_img[y0:y1, x0:x1]
        # print(id_num)
        padded_imgs.append((cropped, (x0,y0,x1,y1), id_num))
    return padded_imgs

IMAGE_DATA = []
for img, accepted in tqdm(ACCEPTED_NOTES, desc="Cropping noteheads", unit="pair"):
    img_arr = np.array(img)
    cropped_images = crop_pad_notes(img_arr, accepted)
    for idx, (crop_img, bbox, id_num) in enumerate(cropped_images):
        if debug:
            plt.figure(figsize=(3,3))
            plt.imshow(crop_img, cmap='gray', vmin=0, vmax=255)
            plt.title(f"Cropped Note {idx+1}")
            plt.axis('off')
            plt.show()
        IMAGE_DATA.append((crop_img, id_num))

TRAINING_IMAGES = [(cv2.resize(crop, (TARGET_SIZE[1], TARGET_SIZE[0])), id_num) for crop, id_num in IMAGE_DATA]

In [ ]:
len(TRAINING_IMAGES)

# Calculating Pitch

In [ ]:
def calculate_pitch(cy, staff_center, unit_size):
    pitch_step = round(-(cy - staff_center) / (unit_size / 2))
    
    pitch_names = {
    -20: "C", -19: "D", -18: "E", -17: "F", -16: "G",
    -15: "A", -14: "B", -13: "C", -12: "D", -11: "E",
    -10: "F",  -9: "G",  -8: "A",  -7: "B",  -6: "C",
     -5: "D",  -4: "E",  -3: "F",  -2: "G",  -1: "A",
      0: "B",   1: "C",   2: "D",   3: "E",   4: "F",
      5: "G",   6: "A",   7: "B",   8: "C",   9: "D",
     10: "E",  11: "F",  12: "G",  13: "A",  14: "B",
     15: "C",  16: "D",  17: "E",  18: "F",  19: "G",
     20: "A"
    }

    pitch = pitch_names.get(pitch_step, "Unknown")
    return pitch, pitch_step

In [ ]:
def calculate_pitch_multi(y, staffs):
    """
    Pick the correct staff for a note at vertical position y,
    then compute its pitch & step using that staff's center & unit_size.
    """
    # 1) find any staff whose bounds contain y
    for st in staffs:
        if st.upper_bound <= y <= st.lower_bound:
            return calculate_pitch(y, st.center, st.unit_size)

    # 2) if none matched perfectly, fall back to the *closest* staff center
    closest = min(staffs, key=lambda s: abs(s.center - y))
    return calculate_pitch(y, closest.center, closest.unit_size)

# Multiclass Classification

## Loading Dataset

In [ ]:
import pandas as pd

df = pd.read_pickle('/kaggle/input/multiclass-noteheads/df_trunc.pkl')

In [ ]:
mapping = {
    "noteheadBlack": 1,
    "noteheadHalf":  0,
    "noteheadWhole": 0
}

df['label_binary'] = df['label'].map(mapping)
print(df['label_binary'].value_counts())

## Building CNN 

This binary classifier will identify if notes are hollow or not

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization)
from tensorflow.keras.callbacks import EarlyStopping

IMAGE_SHAPE = (39, 23, 1)

binary_model = Sequential([
    # First Convolutional Block
    Conv2D(32, (3,3), activation='relu', input_shape=IMAGE_SHAPE),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    
    # Second Convolutional Block
    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    
    # Classifier
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),

    # Single–unit sigmoid output for binary classification
    Dense(1, activation='sigmoid')
])

binary_model.summary()

binary_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
from sklearn.model_selection import train_test_split

X = df["image"].tolist()
X = np.stack(X) # Stacking arrays into a tensor

print("Data shape", X.shape)

y = df["label_binary"].values

X_train, X_test, y_train, y_test = train_test_split(
    X,y,
    test_size = 0.2, # 80/20 split
    stratify=y, # Need equal parts of each class in each
    random_state=42
)
print("Train class distribution:", np.bincount(y_train))
print("Test  class distributino:", np.bincount(y_test))

In [ ]:
result = binary_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=32
)

In [ ]:
probs = binary_model.predict(X_test).ravel() 
predicted = (probs > 0.5).astype(int) 

reverse_mapping = {1:"Not Hollow", 0:"Hollow"}

n = 10
indexes = np.random.choice(len(X_test), size=n, replace=False)

fig, axes = plt.subplots(2, 5, figsize=(15,6))
axes = axes.flatten()

for ax, i in zip(axes, indexes):
    img  = X_test[i].squeeze()
    true_lbl = y_test[i]
    pred_lbl = predicted[i]  

    ax.imshow(img, cmap='gray')
    ax.set_title(
        f"True: {reverse_mapping[true_lbl]}\n"
        f"Pred: {reverse_mapping[pred_lbl]}"
    )
    ax.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
cnn_model.save("binary_model.h5")

### Predicting on images from filtering 

In [ ]:
import cv2

def binarize_and_invert(img, thresh=127):
    _, binary = cv2.threshold(img, thresh, 255, cv2.THRESH_BINARY)
    inverted = 255 - binary
    return inverted

# Apply to your DataFrame
df_images['binarized'] = df_images['image'].apply(binarize_and_invert)

# Quick sanity check on shapes and dtypes
print(df_images[['id', 'binarized']].head())

In [ ]:
X = df_images["binarized"].tolist()
X = np.stack(X) 
X = X[..., np.newaxis].astype("float32") / 255.0

probs = binary_model.predict(X)
preds = (probs > 0.5).astype(int)

preds_flat = preds.ravel() 
id_to_pred = dict(zip(df_images["id"], preds_flat))
id_to_pred = {int(k): int(v) for k, v in id_to_pred.items()}

print(id_to_pred)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

X = np.stack(df_images["binarized"].tolist(), axis=0)
X = X[..., np.newaxis].astype("float32") / 255.0

probs = binary_model.predict(X).ravel()
preds = (probs > 0.5).astype(int)

reverse_mapping = {0:"Hollow", 1:"Not Hollow"}

n = 10
indexes = np.random.choice(len(X), size=n, replace=False)

fig, axes = plt.subplots(2, 5, figsize=(15,6))
for ax, i in zip(axes.flatten(), indexes):
    img = X[i].squeeze()
    pred_lbl = preds[i]
    label_txt = reverse_mapping[pred_lbl]

    ax.imshow(img, cmap='gray')
    ax.set_title(f"{label_txt}\n")
    ax.axis('off')

plt.tight_layout()
plt.show()


# Final Image

In [ ]:
img, notes = ACCEPTED_NOTES [5]
img_pil = Image.fromarray(img)

staff_objects = extract_staffs_from_image(img_pil)

fig, ax = plt.subplots(figsize=(14, 8))
ax.imshow(gray_full, cmap='gray')
ax.set_title("Note Positions with Pitches")

for idx, (((cx, cy), (MA, ma), ang), num_id) in enumerate(notes):
    pred_lbl = reverse_mapping[id_to_pred[num_id]]
    
    pitch, step = calculate_pitch_multi(cy, staff_objects)
    # draw it
    circ = plt.Circle((cx, cy), radius=5, color='red', fill=True)
    ax.add_patch(circ)

    ax.text(cx + 10, cy + 20, f"{pitch}",
            color='blue', fontsize=9, verticalalignment='center')
plt.axis('off')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))
ax.imshow(gray_full, cmap='gray')
ax.set_title("Note Positions with Classification")
for idx, (((cx, cy), (MA, ma), ang), num_id) in enumerate(notes):
    ax.text(cx + 10, cy + 20, f"{id_to_pred[num_id]}",
            color='blue', fontsize=9, verticalalignment='center')

plt.axis('off')
plt.show()

# Appendix

Reference Github: https://github.com/BreezeWhite/oemer

Importing sample image from the [DoReMi](https://github.com/steinbergmedia/DoReMi/?tab=readme-ov-file#OMR-metadata) dataset.